# Download dados do Bucket S3

In [ ]:
import boto3
import botocore
import os
import tarfile

# --- CONFIGURAÇÕES ---
BUCKET_NAME = 'data-architect-test-source'
DOWNLOAD_DIR = 'data_files'
TAR_FILE_NAME = 'ab_test_ref.tar.gz'
RUN_DOWNLOAD = True        # Define se deve baixar os arquivos
RUN_EXTRACTION = True      # Define se deve extrair os arquivos após o download

def download_public_files(bucket_name: str, download_dir: str) -> None:
    """
    Baixa arquivos públicos de um bucket S3 usando acesso anônimo.
    
    Args:
        bucket_name (str): Nome do bucket S3 público.
        download_dir (str): Diretório onde os arquivos serão salvos.
    """
    # Cria cliente S3 com acesso anônimo
    s3 = boto3.client('s3', config=botocore.client.Config(signature_version=botocore.UNSIGNED))
    os.makedirs(download_dir, exist_ok=True)

    # Lista objetos no bucket
    response = s3.list_objects_v2(Bucket=bucket_name)

    if 'Contents' in response:
        for obj in response['Contents']:
            key = obj['Key']
            file_path = os.path.join(download_dir, os.path.basename(key))
            os.makedirs(os.path.dirname(file_path), exist_ok=True)

            print(f'Baixando {key}...')
            s3.download_file(bucket_name, key, file_path)
    else:
        print("Nenhum arquivo encontrado ou acesso negado.")

def extract_tar_gz(tar_path: str, extract_to: str) -> None:
    """
    Extrai arquivos de um arquivo .tar.gz.

    Args:
        tar_path (str): Caminho do arquivo .tar.gz.
        extract_to (str): Pasta de destino para os arquivos extraídos.
    """
    os.makedirs(extract_to, exist_ok=True)

    with tarfile.open(tar_path, "r:gz") as tar:
        tar.extractall(path=extract_to)
        print("Arquivos extraídos:")
        for member in tar.getnames():
            print(f" - {member}")

if __name__ == "__main__":
    if RUN_DOWNLOAD:
        download_public_files(BUCKET_NAME, DOWNLOAD_DIR)

    if RUN_EXTRACTION:
        tar_file_path = os.path.join(DOWNLOAD_DIR, TAR_FILE_NAME)
        extract_tar_gz(tar_file_path, DOWNLOAD_DIR)
